In [139]:
import os
import pandas as pd
import geopandas as gp

In [140]:
shapes = gp.read_file('electoral_precincts/2018Precincts.shp')
shapes = shapes[['geometry']]
shapes.reset_index(inplace=True, drop=True)

part = pd.read_csv('../recent_runoffs/2018_november_cleaned/all_precincts_participation.csv')
part = pd.read_csv('../recent_runoffs/2018_november_cleaned/all_precincts_participation.csv')

res = pd.read_csv('../2020_november/all_precincts_joined/US Senate (Loeffler) - Special.csv')

uni_index = pd.read_csv('unified_precinct_index.csv')

### merge shapes to old runoff participation data

In [141]:
geo_precs = pd.merge(uni_index, shapes, left_on='SHAPE_idx', right_index=True)
part = part[part.columns[3:]]
part = pd.merge(geo_precs, part, left_on='PART_idx', right_index=True, how='left')

part = part[[c for c in part.columns if '_idx' not in c]]

### merge shapes to 2020 election results data

In [142]:
geo_precs = pd.merge(uni_index, shapes, left_on='SHAPE_idx', right_index=True)
res = res[res.columns[2:]]
res = pd.merge(geo_precs, res, left_on='RES_idx', right_index=True)

res = res[[c for c in res.columns if '_idx' not in c]]

# save

In [147]:
target_dir = 'cleaned/2018_precinct_dropoffs/'

# save cols:
t_cols = list(part.columns[:3])+[c for c in part.columns if 'pctChange_nV' in c]+['geometry']


def js_county(county):
    return '-'.join(county.lower().split())

for county in sorted(part.county.unique()):
    t_cols = list(part.columns[:3])+list(part.columns[4:])+['geometry']
    
    c_data = part[part.county==county]
    c_data = c_data[t_cols]
    
    c_data = gp.GeoDataFrame(c_data)
    fpath = os.path.join(target_dir, js_county(county)+'.geojson')
    c_data = gp.GeoDataFrame(c_data)
    c_data.to_file(fpath, driver='GeoJSON')

In [146]:
[c for c in part.columns if 'Black_Male' in c]

['Init_nRegistered_Black_Male',
 'Init_nVoted_Black_Male',
 'Init_pctVoted_Black_Male',
 'Runoff_nRegistered_Black_Male',
 'Runoff_nVoted_Black_Male',
 'Runoff_pctVoted_Black_Male',
 'Delta_nVoted_Black_Male',
 'pctChange_nVoted_Black_Male',
 'Delta_nRegistered_Black_Male',
 'pctChange_nRegistered_Black_Male']